# [0] Imports

In [867]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets

using Dates

# [1] Setup

## [1.1] Parameters

In [868]:
n_jobs = 37
n_vehicles_jobs = n_jobs
n_vehicles_coverage = 10
T = 2 * n_jobs
min_duration = 2
max_duration = 6
speed = 1000/6
coverage_distance = 50
size = 500
step = 50;

## [1.2] Locations, Windows, Loads

In [869]:
Random.seed!(1234)

function create_cluster_sizes(jobs_total)
    Random.seed!(1234)
    jobs_created = 0
    cluster = []
    min_jobs_allowed = n_jobs >= 15 ? 3 : 2
        target_clusters = 5
        while jobs_created != jobs_total
            num_to_add = rand(min(min_jobs_allowed, jobs_total - jobs_created) : 
                min(Int((n_jobs/target_clusters)÷1), jobs_total - jobs_created))
            jobs_created += num_to_add
            push!(cluster, num_to_add)
        end
    
    return cluster
end;

In [870]:
time_windows = []
locations = rand(Uniform(0,size), 1, 2)
work_load = []
cluster = create_cluster_sizes(n_jobs);

In [871]:
function create_time_windows_and_work_load(cluster_sizes, locations)
    Random.seed!(1234)
    locations = rand(Uniform(0,size), 1, 2)
    for size_c in cluster
        first = rand(Uniform(0,size), 1, 2)
        locations = vcat(locations, first)

        job_begins = rand(2:10)
        job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
        push!(time_windows, [job_begins, job_finish])

        time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
        push!(work_load, time_work)

        for neighbour in 1:(size_c-1)
            new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
            new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
            new = hcat(new_x, new_y)
            locations = vcat(locations, new)

            job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
            job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
            push!(work_load, time_work)
        end
    end
    
    return [time_windows, work_load, locations]
end;

In [872]:
time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)
distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];

In [873]:
time_start = datetime2unix(now());

# [2] Helper Methods

## 2.1 Get initial routes

In [874]:
routes = []

for n in 1:n_jobs
    job_route = []
    dist = distances[1, n+1]

    min_t = Int(floor(time_windows[n][1] - dist/speed))
    max_t = Int(ceil(time_windows[n][2] + dist/speed))

    push!(job_route, [[0, min_t], [n, time_windows[n][1]], dist])
    
    for t in time_windows[n][1]:(time_windows[n][2]-1)
        push!(job_route, [[n, t], [n, t+1], 0])
    end
    
    push!(job_route, [[n, time_windows[n][2]], [n_jobs+1, max_t],  dist])
    push!(routes, job_route)
    
end

## 2.2 Compute Cost

In [875]:
function compute_cost(route)
    cost = 0
    for entity in route
        cost += entity[3]
    end
    return cost
end;

In [876]:
C = []
for i in 1:length(routes)
    push!(C, compute_cost(routes[i]))
end

## 2.3 Compute Delta

In [877]:
function compute_delta(routes)
    Q = length(routes)
    delta = [[[0.0 for q in 1:Q] for t in 1:T] for i in 1:n_jobs]
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1, time1 = arc[1]
            loc2, time2 = arc[2]
            if (loc1 != 0) & (loc1 != n_jobs + 1)
                delta[loc1][time1][rindex] = 1
            end
            
            if (loc2 != 0) & (loc2 != n_jobs + 1)
                delta[loc2][time2][rindex] = 1
            end
        end
    end
    return delta
end;

In [878]:
delta = compute_delta(routes);

## 2.4 Compute U

In [879]:
function compute_u(routes)
    Q = length(routes)
    u = [[0 for q in 1:Q] for i in 1:n_jobs]
    
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1 = Int(arc[1][1])
            loc2 = Int(arc[2][1])
            if (1 <= loc1 <= n_jobs)
                u[loc1][rindex] = 1
            end
            if (1 <= loc2 <= n_jobs)
                u[loc2][rindex] = 1
            end
        end
    end
    return u
end;

In [880]:
u = compute_u(routes);

## 2.5 Label-Setting Algorithm

In [881]:
function sp_lsa(n, travel_distance, travel_time, windows, load, rho_v, pi_v)
    N = [[1]] 
    Times = [ [0] ] 
    R = [rho_v]  
    NRC = [Inf for i in 1:n]

    current_state = 1
    total_state = 1

    while 1==1
        cur_path = N[current_state]
        last_node_cur = last(cur_path)

        # STEP 1: Check whether we want to move on to checking the next path
        if (last_node_cur == n+2)
            current_state += 1
            if current_state > total_state
                # print(current_state, " ", total_state, " nothing more to check")
                break
            else
                continue
            end
        end

        # STEP 2: Enter the for loop.
        for i in 2:(n+2)

            # STEP 2-1: Check that the current node is not already inside our path.
            if ~(i in cur_path)

                cur_time = last(Times[current_state])
                dist_nec = travel_time[last_node_cur, i]
                old_job_time_nec = last_node_cur > 1 ? load[last_node_cur-1] : 0
                new_job_time_nec = load[i-1]
                new_window_start, new_window_close = windows[i-1]

                if last_node_cur != 1
                    if cur_time + dist_nec + old_job_time_nec + new_job_time_nec > new_window_close
                        continue
                    end
                end

                reach_new_node_time = max(cur_time + dist_nec + old_job_time_nec, new_window_start)

                # SECOND CHECK
                add_segdist = travel_distance[last_node_cur, i]
                subtract_pi = i < n+2 ? -pi_v[i-1] : 0

                proposed_cost = R[current_state] + add_segdist + subtract_pi
                
                if i < n+2
                    if proposed_cost > NRC[i-1]
                        continue
                    else
                        NRC[i-1] = proposed_cost
                    end
                end

                # STEP 2-3: Add the path to our N.
                push!(N, copy(cur_path))
                push!(N[total_state+1], i)

                new_times_array = copy(Times[current_state])
                push!(new_times_array, reach_new_node_time)
                push!(Times, new_times_array)

                push!(R, proposed_cost)

                total_state += 1

            end
        end

        current_state += 1 # concluded iteration for that state in mind. Now next one

        #= It's possible that we've just caught up to the final total state, meaning we didn't generate
        anything and can stop here. =#
        if current_state == total_state
            break
        end     
    end

    return N, R, Times
end;

### 2.5.1 Label-Setting Algorithm Constant Variables

In [882]:
distances_label = deepcopy(distances)
distances_label = hcat(distances_label, distances_label[:, 1])
distances_label = vcat(distances_label, collect(push!(distances_label[:, 1], 0)'));

In [883]:
travel_times = ceil.(distances_label / speed);

In [884]:
windows_label = deepcopy(time_windows)
push!(windows_label, [0, 100]);

In [885]:
load_label = deepcopy(work_load)
push!(load_label, 0);

## 2.6 Extract Best Route in SP

In [886]:
function extract_best_route(sub_paths, sub_rc, sub_times, n_jobs)

    best_rc = Inf
    best_index = 0

    for i in 1:length(sub_paths)
        if sub_rc[i] < best_rc && last(sub_paths[i]) == n_jobs + 2
            best_rc = sub_rc[i]
            best_index = i
        end
    end

    return sub_paths[best_index], sub_rc[best_index], sub_times[best_index]
end;

## 2.7 Generate full time info on route

In [887]:
function generate_full_time_info(best_route, best_times, need_to_pad=true)
    best_route_info = []
    for locindex in 1:length(best_route)-1
        #=
        Strategy:
        If you are not at a job location, you don't need to capture
        stationary information. Otherwise you do.

        Anyways we want data moving from one to the next.
        =#
        cur_loc = best_route[locindex]
        cur_time = best_times[locindex]
        time_to_start_moving = best_times[locindex]
        
        cur_job_num = need_to_pad ? cur_loc - 1 : cur_loc # delete padding!

        #stationary data comes first.
        if 1 < locindex < n_jobs + 2
            for time in cur_time:cur_time+work_load[cur_job_num]-1
                route_detail = [ [cur_job_num, time], [cur_job_num, time+1], 0]
                push!(best_route_info, route_detail)
            end

            time_to_start_moving += work_load[cur_job_num]
        end

        new_loc = best_route[locindex+1]
        new_loc_pad = need_to_pad ? new_loc - 1 : new_loc
        #then if we move, we move.
        route_detail = [ [cur_job_num, time_to_start_moving], [new_loc_pad, best_times[locindex+1]], distances_label[cur_job_num+1, new_loc_pad + 1]]
        push!(best_route_info, route_detail)
    end

    return best_route_info
end;

# [3] Column Generation Loop

In [888]:
modelcg = Model(Gurobi.Optimizer);
set_optimizer_attribute(modelcg, "OutputFlag", 0);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [889]:
keep_going = true

num_routes_at_a_time = 1
z_values = []
OBJ = 0

while keep_going
    Q = length(routes)
    
    unregister(modelcg, :z)
    @variable(modelcg, 0 <= z[1:Q] <= 1) # should be Bin
    
    unregister(modelcg, :unique)
    @constraint(modelcg, unique[i in 1:n_jobs], sum(u[i][q] * z[q] for q in 1:Q) == 1);
    
    unregister(modelcg, :driver)
    @constraint(modelcg, driver, sum(z[q] for q in 1:Q) <= n_vehicles_jobs)
    
    @objective(modelcg, Min, sum(C[q] * z[q] for q in 1:Q));
    
    optimize!(modelcg)
    OBJ = objective_value(modelcg)
    
    push!(z_values, round.(value.(z), digits = 3))
    
    pi_values = dual.(unique)
    rho_value = dual.(driver)
    
    sub_paths, sub_rc, sub_times = sp_lsa(n_jobs, distances_label, travel_times, windows_label, load_label, rho_value, pi_values)
    
    best_route, best_rc, best_times = extract_best_route(sub_paths, sub_rc, sub_times, n_jobs)
    
    if best_rc > -1e-6
        keep_going = false
        break
    end
    
    push!(routes, generate_full_time_info(best_route, best_times))
    
    u = compute_u(routes)
    delta = compute_delta(routes);
    

    C = []
    for i in 1:length(routes)
        push!(C, compute_cost(routes[i]))
    end;

end

# [4] Extract CG Data

## [4.1] Time used and objective

In [890]:
println("OPT USED: ", datetime2unix(now()) - time_start)

OPT USED: 2.700000047683716


In [891]:
println(OBJ)

3761.0669898154656


## [4.2] Get meaningful routes, nodes, and times

In [892]:
acc_routes = []

nodes_only = []
times_only = []

assoc_z_values = []
for i in n_jobs+1:length(routes)
    if last(z_values)[i] > 1e-4
        push!(acc_routes, routes[i])
        nodes_visited = []
        times_visited = [0,]
        for entity in routes[i]
            place = entity[1][1]
            if ~(place in nodes_visited)
                push!(nodes_visited, place)
            end
            time = entity[2][2]
            if entity[3] > 0.0
                push!(times_visited, time)         
            end
        end
        push!(nodes_visited, n_jobs+1)
        push!(nodes_only, nodes_visited)
        push!(times_only, times_visited)
        push!(assoc_z_values, last(z_values)[i])
    end
end

## [4.3] Display z-values

In [893]:
#=
for index in 1:length(assoc_z_values)
    println(nodes_only[index])
    println(times_only[index])
    println(assoc_z_values[index])
    println()
end
=#

Now, routes with z-value > 0 are stored in `acc_routes`; nodes in `nodes_only` and times in `times_only`.

## [4.4] Test for loner nodes

In [894]:
loner_nodes = []
for job_num in 1:n_jobs
    job_included = false
    for route in nodes_only
        if job_num in route
            job_included = true
        end
    end
    
    if ~job_included
        println("Job ", job_num, " was not in a fancy route.")
        push!(loner_nodes, job_num)
    end
end

If we have loner nodes, we have to go back and add it.

I know it's slightly less efficient than if [4.3] just added all routes, or even non-trivial routes (by checking that the length of the route > 1), but loner nodes are very bad and rarely occur.

Also, I think distinguishing loner nodes in code will bring more attention to it, as it deserves.

In [895]:
for loner_node in loner_nodes
    push!(acc_routes, routes[loner_node])
end

## [4.5] Get imperfect routes

In [896]:
imperfect_routes = []
imperfect_nodes = []
imperfect_times = []

for index in 1:length(assoc_z_values)
    if assoc_z_values[index] < 1
        push!(imperfect_routes, acc_routes[index])
        push!(imperfect_nodes, nodes_only[index])
        push!(imperfect_times, times_only[index])
    end
end

# [5] Setup for Set Partitioning for Integer Programming

Only need to do this if there are non-integer z-values

## [5.1] Process Routes and Generate Hybrids

### [5.1.1] Algorithm to Generate Suffixes from Common Stem

In [897]:
function generate_route_suffixes(n, stem_nodes, stem_times, candidates, distances, travel_times, windows, loads)
    #=
    Given a route stem and a set of candidates for job appendages,
    provide ALL valid full routes that have valid suffixes.
    
    For instance, suppose we had a stem 
    00 16 17 18 19
    with times
    00 02 08 12 17
    
    and our candidates were 13, 14, 15
    
    then we would see which of these could be tacked on to the route before
    going back to the depot.
    
    This reads the same as our original LSA for generating all routes,
    it's just that we limit our set of possibilities.
    =#
    
    # first, include the depot
    if ~(n+1 in candidates)
        push!(candidates, n+1)
    end
    
    N = [ stem_nodes ]
    T = [ stem_times ]
    # cost is not a concern, we evaluate separately
    
    current_state = 1
    total_state = 1
    
    while true
        cur_path = N[current_state]
        last_cur_node = last(cur_path)
        
        # check if we can move on
        if (last_cur_node == n+1)
            current_state += 1
            if current_state > total_state
                break
            else
                continue
            end
        end
        
        for candidate in candidates
            # so candidate is a job number
            
            if ~(candidate in cur_path)
                # do feasibility check, we know our last state wasn't 0
                cur_time = last(T[current_state])
                dist_nec = travel_times[last_cur_node+1, candidate+1]
                old_job_time_nec = last_cur_node != 0 ? loads[last_cur_node] : 0
                new_job_time_nec = loads[candidate]
                new_window_start, new_window_close = windows[candidate]

                #if invalid, don't bother
                if cur_time + dist_nec + old_job_time_nec + new_job_time_nec > new_window_close
                    continue
                end
                
                push!(N, copy(cur_path))
                push!(N[total_state+1], candidate)
                
                new_times = copy(T[current_state])
                push!(new_times, max(cur_time + dist_nec + old_job_time_nec, new_window_start))
                push!(T, new_times)
                
                total_state += 1
            end
        end
        
        current_state += 1
        
        if current_state == total_state
            break
        end
    end
    
    return N, T
end;

### [5.1.2] Determine stems and blenders

In [898]:
function find_index(thing, array)
    # Julia's python index() equivalent, indexin, is garbage!
    for i in 1:length(array)
        if thing == array[i]
            return i
        end
    end
    return -1
end;

In [899]:
function set(array)
    #= Get the unique elements. Equivalent to unique() but 
    Julia doesn't want to let it work because allegedly
    some of my arrays are vectors of constraints, 
    even though they are clearly just ints >:-(
    =#
    answer = []
    for thing in array
        if ~(thing in answer)
            push!(answer, thing)
        end
    end
    return answer
end;

In [900]:
function minimum(array)
    # Julia can't min on vectors or matrices...
    smallest = array[1]
    for item in array
        if item < smallest
            smallest = item
        end
    end
    return smallest
end;

In [901]:
function stems_and_blenders(list_nodes, list_times = [])
    #=
    Given a set of route nodes list and their corresponding
    times for which the z-value is less than 1,
    determine the stems and blenders among them.
    
    The full algorithm is described in the document
    "Stemming Techniques for Generating Hybrid Routes.pdf".
    
    For example, if our non-z=1-route-nodes were
    1 5 9 3 8
    1 5 6 2 3
    1 5 6 3
    2 4 9 6
    2 4 9 8
    
    then return [
    [ [1 5], [1_time, 5_time], [2, 3, 6, 8, 9]],
    [ [2 4 9], [2_time 4_time 9_time], [6 8]]
    ]
    
    To debug or have fun I give the option to not have times.
    =#
    
    # STEP 1: obtain all stems.
    all_stems = set([list_nodes[i][1] for i in 1:length(list_nodes)])
    
    # STEP 2: determine which routes belong with that stem.
    route_belong = Dict(i => [] for i in all_stems)
    for nroute in list_nodes
        push!(route_belong[nroute[1]], nroute)
    end
    
    # STEP 3: elongate the stem as much as possible.
    
    answer = []
    
    for stem_key in keys(route_belong)
        stem_elts = []
        stem_times = []
        blender = []
        
        all_appl_routes = route_belong[stem_key]
        
        highest_stem_index = 1
        
        max_stem_index = minimum([length(all_appl_routes[i]) for i in 1:length(all_appl_routes)])
        
        while highest_stem_index < max_stem_index
            candidates = [all_appl_routes[i][highest_stem_index + 1] for i in 1:length(all_appl_routes)]
            cand_indices = set(candidates)
            if length(cand_indices) > 1
                break
            else
                highest_stem_index += 1
            end
        end
                
        # highest_stem_index now has the highest valid stem index
        stem_elts = all_appl_routes[1][1:highest_stem_index]
        stem_times = length(list_times) > 0 ? list_times[find_index(all_appl_routes[1], list_nodes)][1:highest_stem_index] : []
        
        # STEP 4: determine the blender
        blender = []
        for route in all_appl_routes
            for elt in route[highest_stem_index+1:end]
                push!(blender, elt)
            end
        end
        blender = set(blender)

        # STEP 5: put it all together
        push!(answer, [stem_elts, stem_times, blender])
    end
    
    return answer
end;

### [5.1.3] Determine stems and blenders

In [902]:
# don't want to include depot
want_nodes = [imperfect_nodes[i][2:end-1] for i in 1:length(imperfect_nodes)];
want_times = [imperfect_times[i][2:end-1] for i in 1:length(imperfect_times)];

In [903]:
stems_blenders = stems_and_blenders(want_nodes, want_times)

2-element Vector{Any}:
 Vector{Any}[[36, 37, 29, 30, 31], [2, 8, 12, 18, 28], [33, 34, 35]]
 Vector{Any}[[1, 2, 3, 4, 32], [5, 13, 20, 30, 40], [33, 34, 35]]

Critical note: Does not include the 0's at the front!

### [5.1.4] Helper functions

In [904]:
function get_routes_from_nodes_and_times(nodes, times)
    new_valid_routes = []

    for i in 1:length(nodes)
        push!(new_valid_routes, generate_full_time_info(nodes[i], times[i], false))
    end
    
    return new_valid_routes
end;

In [905]:
function get_depot_ending(new_nodes, new_times)
    #= After the new LSA, not every path is usable.
    Only get those that lead us to the finish line.
    =#
    new_valid_nodes = []
    new_valid_times = []
    seen = []

    for i in 1:length(new_nodes)
        if last(new_nodes[i]) == n_jobs + 1 && ~(new_nodes[i] in seen)
            push!(new_valid_nodes, new_nodes[i])
            push!(new_valid_times, new_times[i])
            push!(seen, new_nodes[i])
        end
    end
    return new_valid_nodes, new_valid_times
end;

### [5.1.5] Add to routes to consider

In [906]:
for i in 1:length(stems_blenders)
    routing_nodes, routing_times, blenders = stems_blenders[i]
    insert!(routing_nodes, 1, 0)
    insert!(routing_times, 1, 0)
    new_nodes, new_times = generate_route_suffixes(n_jobs, routing_nodes, routing_times, blenders, distances_label, travel_times, windows_label, load_label)
    good_nodes, good_times = get_depot_ending(new_nodes, new_times)
    valid_new_routes = get_routes_from_nodes_and_times(good_nodes, good_times)
    
    for route in valid_new_routes
        push!(acc_routes, route)
    end
end

Add trivial stay-at-depot route

In [907]:
depot_stay = [ [ [0, 0], [n_jobs+1, 0], 0],  ]
push!(acc_routes, depot_stay);

In [908]:
acc_n_routes = length(acc_routes)

26

In [909]:
acc_C = []
for i in 1:acc_n_routes
    push!(acc_C, compute_cost(acc_routes[i]))
end

In [910]:
acc_u = compute_u(acc_routes);

## [5.2] Create Set Partitioning for IP

In [911]:
model_set = Model(Gurobi.Optimizer);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [912]:
@variable(model_set, z[1:n_vehicles_jobs, 1:acc_n_routes], Bin);

In [913]:
@objective(model_set, 
    Min, 
    sum(sum(z[k,q]*acc_C[q] for q in 1:acc_n_routes) for k in 1:n_vehicles_jobs))
;

In [914]:
@constraint(model_set, 
    unique[i in 1:n_jobs],
    sum(sum(z[k,q]*acc_u[i][q] for q in 1:acc_n_routes) for k in 1:n_vehicles_jobs) == 1);

@constraint(model_set, 
    driver[k in 1:n_vehicles_jobs], 
    sum(z[k,q] for q in 1:acc_n_routes) == 1);

In [915]:
optimize!(model_set)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 74 rows, 962 columns and 6660 nonzeros
Model fingerprint: 0x089d0847
Variable types: 0 continuous, 962 integer (962 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 8e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3765.1967149
Presolve removed 26 rows and 184 columns
Presolve time: 0.01s
Presolved: 48 rows, 778 columns, 2298 nonzeros
Variable types: 0 continuous, 778 integer (778 binary)

Root relaxation: objective 3.751495e+03, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3751.4949192 3751.49492  0.00%     -    0s

Explored 1 nodes (29 simplex 

## [5.3] Extract valid routes

In [916]:
function extract_route_nodes(route)
    seen = []
    answer = []
    for route_segment in route
        first_node = route_segment[1][1]
        second_node = route_segment[2][1]
        if ~(first_node in seen)
            push!(seen, first_node)
            push!(answer, first_node)
        end
        if ~(second_node in seen)
            push!(seen, second_node)
            push!(answer, second_node)
        end
    end
    return answer
end;

In [917]:
acc_z = value.(z);

In [918]:
for non_trivial_route_index in 1:acc_n_routes-1
    # don't want the last one which lots of vehicles will have
    used_times = sum(acc_z[:, non_trivial_route_index])
    if used_times >= 1.0
        nodes_traversed = extract_route_nodes(acc_routes[non_trivial_route_index])
        println("Route ", nodes_traversed, " was used ", used_times, " times!")
    end
end

Route Any[0, 14, 15, 16, 17, 18, 19, 20, 38] was used 1.0 times!
Route Any[0, 5, 6, 7, 8, 9, 38] was used 1.0 times!
Route Any[0, 10, 11, 12, 13, 38] was used 1.0 times!
Route Any[0, 21, 22, 23, 24, 28, 38] was used 1.0 times!
Route Any[0, 1, 2, 3, 4, 32, 33, 34, 35, 38] was used 1.0 times!
Route Any[0, 25, 26, 27, 38] was used 1.0 times!
Route Any[0, 36, 37, 29, 30, 31, 38.0] was used 1.0 times!


In [919]:
println("OBJECTIVE: ", objective_value(model_set))

OBJECTIVE: 3751.494919249296
